In [1]:
import random
import operator
import csv
import itertools

import numpy

#---------------import for plotting------------------
import matplotlib.pyplot as plt
import networkx as nx
import pydot
from networkx.drawing.nx_pydot import graphviz_layout
#----------------------------------------------------

from deap import alteralgorithms as algorithms
from deap import base
from deap import creator
from deap import tools
from deap import altergp as gp


# Read csv file and puts it in a list of lists as string type.
with open("iris.csv") as matrixbase:
    matrixReader = csv.reader(matrixbase)
    matrixDB_str = list(list(elem for elem in row) for row in matrixReader)


#Cast all the fields of the string matrix to the correct types
matrixDB=[]
for row in matrixDB_str:
    o=[]

    for elem in row:
        try:
        	o.append(int(elem)) #elem = int
        except ValueError:
        	try:
        		o.append(float(elem)) #elem = float
        	except ValueError:
        		o.append(str(elem)) #elem = string

    matrixDB.append(o)


#Matrix of variables which has Min and Max per col and finally var "o" colect all types of possibles output
varMatrix=[]
for n in range(len(matrixDB[0])):
    o=[]
    
    if n < (len(matrixDB[0])-1):
    	for elem in matrixDB:
    		o.append(*elem[n:n+1])
    	varMatrix.append([numpy.max(o), numpy.min(o)])
    
    else:
    	for elem in matrixDB:
    		o.append(*elem[n:n+1]) #array of all outputs but it has duplicate values


#Remove duplicates of var "o" and var "outputs" gets the exacts outputs of the database
outputs = list(dict.fromkeys(o))
#min and max value of all input variables
maxTotal = numpy.max(varMatrix)
minTotal = numpy.min(varMatrix)


#iterationsMatrix is an array of Matrix which contains all splits removing each time 1 variable that it will be used to compare one clases vs others.
#True value assigned to class that will be evaluate first
#False to class that will be evaluate after
iterationsMatrix=[]
if len(outputs) > 2 and type(outputs[0]) == str:
    for n in range(len(outputs)-1):
        matrix=[]

        for row in matrixDB:
            o=[]
            if not (row[len(matrixDB[0])-1] in outputs[0:n]): #remove the previous class assigned to True
                for elem in row:
                    if type(elem) == str and elem == outputs[n]:
                        o.append(bool(True))
                    else:
                        if type(elem) == str:
                            o.append(bool(False))
                        else:
                            o.append(elem)
                matrix.append(o)
        iterationsMatrix.append(matrix)
else:
    iterationsMatrix.append(matrixDB)


# defined a new primitive set for strongly typed GP
pset = gp.PrimitiveSetTyped("MAIN", itertools.repeat(type(matrixDB[0][0]), len(matrixDB[0])-1), bool, "V")


# boolean operators
pset.addPrimitive(operator.and_, [bool, bool], bool, "And")
pset.addPrimitive(operator.or_, [bool, bool], bool, "Or")

# logic operators
class ERC(object): pass
pset.addPrimitive(operator.lt, [float, ERC], bool)
pset.addPrimitive(operator.gt, [float, ERC], bool)
pset.addPrimitive(operator.le, [float, ERC], bool)
pset.addPrimitive(operator.ge, [float, ERC], bool)
pset.addPrimitive(operator.eq, [float, ERC], bool)
pset.addPrimitive(operator.ne, [float, ERC], bool)

def In(input, leftmost, rightmost):
    if leftmost > rightmost:
        aux = leftmost
        leftmost = rightmost
        rightmost = aux
    
    if leftmost < input and input < rightmost:
        return True
    else:
        return False

def Out(input, leftmost, rightmost):
    if leftmost > rightmost:
        aux = leftmost
        leftmost = rightmost
        rightmost = aux
    
    if leftmost < input and input < rightmost:
        return False
    else:
        return True

pset.addPrimitive(In, [float, ERC, ERC], bool)
pset.addPrimitive(Out, [float, ERC, ERC], bool)

# terminals
pset.addEphemeralConstant("rand", lambda: random.uniform(minTotal, maxTotal), ERC)
pset.addTerminal(True, bool)
pset.addTerminal(False, bool)

# Define the operator fitnessMax and create the individual who uses it
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genHalfAndHalf, pset=pset, min_=1, max_=6)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("compile", gp.compile, pset=pset)

def evalIrisbase(individual, matrix):
    # Transform the tree expression in a callable function
    func = toolbox.compile(expr=individual)
    #print (str(individual))
    #print("")
    result = sum(bool(func(*elem[:len(matrixDB[0])-1])) is bool(elem[len(matrixDB[0])-1]) for elem in matrix)

    return result,
    
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("expr_mut", gp.genFull, min_=1, max_=6)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr_mut, pset=pset)

toolbox.decorate("mate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))
toolbox.decorate("mutate", gp.staticLimit(key=operator.attrgetter("height"), max_value=17))


def plotting(individual):
    nodes, edges, labels = gp.graph(individual)
    g = nx.Graph()
    g.add_nodes_from(nodes)
    g.add_edges_from(edges)
    pos = graphviz_layout(g, prog="dot")

    nx.draw_networkx_nodes(g, pos)
    nx.draw_networkx_edges(g, pos)
    nx.draw_networkx_labels(g, pos, labels)
    plt.show()


def train(matrix):
    pop = toolbox.population(n=500)
    hof = tools.HallOfFame(1)

    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", numpy.mean)
    stats.register("std", numpy.std)
    stats.register("min", numpy.min)
    stats.register("max", numpy.max)
    
    toolbox.register("evaluate", evalIrisbase, matrix=matrix)

    algorithms.eaSimple(pop, toolbox, 0.5, 0.2, 100, stats, halloffame=hof, verbose=True, reportFrec=10)

    return pop, stats, hof,

def main():
    random.seed(40)

    results = []
    for x in range(len(iterationsMatrix)):
        print ("----- Iteration", x+1, "-----")

        pop, stats, hof = train(iterationsMatrix[x])       
        results.append([pop, stats, hof])

    print ("----- HallOfFame -----")
    for x in range(len(results)+1):
        if x == 0:
            print ("Rule", x+1,": IF (", str(results[x][2][0]), ") THEN ( Class =", outputs[x], ")" )
        else:
            if x != len(results): 
                print ("Rule", x+1,": ELSE IF (", str(results[x][2][0]), ") THEN ( Class =", outputs[x], ")" )
            else:
                print ("Rule", x+1,": ELSE ( Class =", outputs[x], ")" )



if __name__ == "__main__":
    main()

----- Iteration 1 -----
gen	nevals	avg   	std    	min	max
0  	500   	73.796	33.8613	0  	150
10 	296   	124.702	41.0835	0  	150
20 	284   	135.586	31.2933	0  	150
30 	294   	137.756	29.176 	0  	150
40 	309   	138.328	29.1194	0  	150
50 	303   	139.532	27.323 	0  	150
60 	267   	138.5  	29.4424	0  	150
70 	308   	139.234	29.0267	2  	150
80 	304   	139.644	28.061 	12 	150
90 	299   	138.534	28.1026	26 	150
100	286   	139.624	28.1756	9  	150
----- Iteration 2 -----
gen	nevals	avg 	std    	min	max
0  	500   	50.9	9.24673	7  	94 
10 	287   	78.776	24.7135	6  	97 
20 	290   	88.748	16.7843	21 	97 
30 	279   	88.39 	17.6547	8  	97 
40 	322   	88.814	16.7759	6  	97 
50 	257   	88.676	17.2681	8  	97 
60 	313   	88.908	16.6966	8  	97 
70 	283   	89.414	15.9044	45 	97 
80 	261   	89.824	15.2759	27 	97 
90 	293   	89.73 	15.0928	27 	97 
100	301   	89.626	15.8327	16 	97 
----- HallOfFame -----
Rule 1 : IF ( lt(V3, 0.6642805312990222) ) THEN ( Class = Iris-setosa )
Rule 2 : ELSE IF ( And(Out(V3, 1.69